In [15]:
import pandas as pd
import numpy as np

In [471]:
vals = []
with open('./input.txt', 'r') as file:
    for line in file:
        vals.append(line.strip())
        
vals[:5]

['Sensor at x=3772068, y=2853720: closest beacon is at x=4068389, y=2345925',
 'Sensor at x=78607, y=2544104: closest beacon is at x=-152196, y=4183739',
 'Sensor at x=3239531, y=3939220: closest beacon is at x=3568548, y=4206192',
 'Sensor at x=339124, y=989831: closest beacon is at x=570292, y=1048239',
 'Sensor at x=3957534, y=2132743: closest beacon is at x=3897332, y=2000000']

In [436]:
test ='''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3'''

vals = []
for line in test.split('\n'):
    vals.append(line.strip())

vals[:5]



['Sensor at x=2, y=18: closest beacon is at x=-2, y=15',
 'Sensor at x=9, y=16: closest beacon is at x=10, y=16',
 'Sensor at x=13, y=2: closest beacon is at x=15, y=3',
 'Sensor at x=12, y=14: closest beacon is at x=10, y=16',
 'Sensor at x=10, y=20: closest beacon is at x=10, y=16']

In [472]:
sensors = []
beacons = []
distances = []
for v in vals:

    split = v.split(' ')
    sensor_x = int(split[2].split('=')[-1][:-1])
    sensor_y = int(split[3].split('=')[-1][:-1])
    beacon_x = int(split[-2].split('=')[-1][:-1])
    beacon_y = int(split[-1].split('=')[-1])
    sensors.append([sensor_y, sensor_x])
    beacons.append([beacon_y, beacon_x])  
    
    distance = np.abs(beacon_x-sensor_x)+np.abs(beacon_y-sensor_y)
    assert int(distance)==distance
    distance = int(distance)  
    distances.append(distance)
    
beacons = np.array(beacons, dtype=int)
sensors = np.array(sensors, dtype=int)
distances = np.array(distances, dtype=int)

row10 = []

In [215]:



target = 10# 2000000 #10#
row10=[]
for idx, (b,s,distance) in enumerate(zip(beacons, sensors, distances)):
    print(idx)
    
    
    if s[0]+distance < target or s[0]-distance > target:
        continue
        
    
    for i in range(-distance, distance+1):
        if i+s[0]==target:
            for j in range(-distance, distance+1):
            
                if np.abs(i)+np.abs(j)<=distance:
                    if not ((sensors==[i+s[0], j+s[1]]).all(1)).any() and not ((beacons==[i+s[0], j+s[1]]).all(1)).any():

                        row10.append((i+s[0], j+s[1]))
#    if idx==3:
#        assert 3==2
#            break

len(set(row10))

0
1
2
3
4
5
6
7
8
9
10
11
12
13


26

In [267]:
min_x = 0
max_x = 4_000_000

for target in range(max_x+1):
    row10=np.zeros(max_x+1)
    for idx, (b,s,d) in enumerate(zip(beacons, sensors, distances)):
        if target+d-s[0] > 0: 
            i = np.abs(target-s[0])
            row10[np.arange(np.max([min_x, s[1]-d+i]),np.min([max_x, d+s[1]-i])+1)]=1
            
    if len(np.argwhere(row10==0))>0:
        print(f"x={np.argwhere(row10==0)},y={target}")
        break

x=[[     27]
 [     28]
 [     29]
 ...
 [3999998]
 [3999999]
 [4000000]],y=0


In [97]:
x = np.arange(np.max([min_x, s[1]-d+i]),np.min([max_x, d+s[1]-i])+1)
y = target
np.abs(y-s[0])

In [196]:
s[0]+d, target

(5, 0)

In [273]:
target+distances-sensors[:,0]>0

array([False, False, False, False, False, False, False,  True, False,
        True, False, False,  True, False, False, False, False, False,
       False,  True,  True, False, False, False, False, False])

In [484]:
min_x = 0
max_x = 4_000_000
big = set(np.arange(max_x+1))
for target in big:
    if target%1_000_000==0:
        print(target)


    idx = target+distances-sensors[:,0]>0      # Is the current y location reachable?
    lower = sensors[idx][:,1]-distances[idx]+np.abs(target-sensors[idx][:,0]) #lower
    upper = sensors[idx][:,1]+distances[idx]-np.abs(target-sensors[idx][:,0]) #upper

    idx = lower<=upper  
    bounds = np.stack([lower[idx], upper[idx]])
    bounds = np.where(bounds<min_x,min_x,bounds)  
    bounds = np.where(bounds>max_x,max_x,bounds)
    
    
    bounds.sort()
    ans = bounds.T[0]
    for b in bounds.T[1:]:
        #     x----y
        #   x---------y
        if b[0]<=ans[0] and b[1]>=ans[1]:
            ans=b
        #     x----y
        #       x------y
        elif b[1]>=ans[1] and b[0]<=ans[1]:
            ans[1]=b[1]
        #     x----y
        #  x----y            
        elif b[0]<=ans[0] and b[1]<=ans[1]:
            ans[0]=b[0]

        #   x----------y
        #      x--y
        elif b[0]>=ans[0] and b[1]<=ans[1]:
            continue
        # x-----y
        #         x----y
        # WINNING
        else:
            
            assert 3==2, f"({ans[1]+1}, {target}) which is {(ans[1]+1)*4000000+target}"

     
            

0
1000000
2000000
3000000


AssertionError: (3403960, 3289729) which is 13615843289729

In [482]:
(3403959+1)*4000000+target

13615843289729